In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import textstat
import stanza 
import re 

# Load up SWC

This loads up sessions with clicks, starts extracting lexical characteristics, and extras all unique queries for more feature extraction.

In [2]:
searchFeatSWC = pickle.load( open( "Pickles/SearchFeatSWC.p", "rb" ) )
searchFeatSQS = pickle.load( open( "Pickles/SearchFeatSQS.p", "rb" ) )
vocabFeat = pickle.load( open( "Pickles/VocabFeat.p", "rb" ) )
lexFeat = pickle.load( open( "Pickles/LexFeat.p", "rb" ) )
synFeat = pickle.load( open( "Pickles/SynFeat.p", "rb" ) )
sPFeat = pickle.load( open( "Pickles/SPFeat.p", "rb" ) )

In [3]:
searchFeatSWC[searchFeatSWC['numQueries'] == 0]

,sID,query,timestamp,click,website,type,order,class,queryDistance,timeQueries,...,allSameQueries,repeatClicks,uniqueClicks,allSameClicks,timeClicks,clickDistance,numQueries,numClicks,meanClickPosition,numClicksPerQuery


In [4]:
searchFeatSWC.groupby('sID').count()

,query,timestamp,click,website,type,order,class,queryDistance,timeQueries,repeatQueries,...,allSameQueries,repeatClicks,uniqueClicks,allSameClicks,timeClicks,clickDistance,numQueries,numClicks,meanClickPosition,numClicksPerQuery
sID,,,,,,,,,,,,,,,,,,,,,
0,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
1,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
3,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39895,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,0,3,3,3,3,3
39896,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,0,8,8,8,8,8
39897,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,0,3,3,3,3,3


In [5]:
sPFeat = sPFeat.merge(vocabFeat)
synFeat = synFeat.merge(lexFeat)
searchFeat = synFeat.merge(sPFeat)

In [6]:
SWCAll = searchFeatSWC.merge(searchFeat, how='inner', on='query')

In [7]:
SQSAll = searchFeatSQS.merge(searchFeat, how='inner', on='query')

In [8]:
SWCAll[SWCAll['numQueries'] == 0]

,sID,query,timestamp,click,website,type,order,class,queryDistance,timeQueries,...,com,net,org,edu,gov,http,AND,OR,quotes,inter


In [9]:
pickle.dump(SWCAll, open( "DataSets/SWCFeatures/SWCFeat.p", "wb" ) )
pickle.dump(SQSAll, open( "DataSets/SQSFeatures/SQSFeat.p", "wb" ) )

In [10]:
SWCQ = SWCAll[SWCAll['type'] == 'Q'].groupby('sID').mean()
SWCQ = SWCQ.drop(columns = ['order','queryDistance','timeQueries','repeatQueries','repeatClicks','clickDistance','meanClickPosition',
    'numClicks', 'numClicksPerQuery', 'numQueries', 'timeClicks', 'uniqueQueries','allSameClicks','uniqueClicks','allSameQueries']) 

In [11]:
SWCC = SWCAll.groupby('sID').mean()
SWCC = SWCC[['repeatClicks','clickDistance','meanClickPosition',
   'numClicks', 'numClicksPerQuery', 'numQueries', 'timeClicks', 'uniqueQueries','allSameClicks','uniqueClicks','allSameQueries']]

In [12]:
SWCC[SWCC['numClicks']==0]

,repeatClicks,clickDistance,meanClickPosition,numClicks,numClicksPerQuery,numQueries,timeClicks,uniqueQueries,allSameClicks,uniqueClicks,allSameQueries
sID,,,,,,,,,,,


In [13]:
SWCCQ = SWCAll[SWCAll['type'] == 'Q'].groupby('sID')[['queryDistance','timeQueries','repeatQueries']].mean()

In [14]:
SWCC = SWCC.join(SWCCQ)

In [15]:
SWCAll = SWCQ.join(SWCC).fillna(0)

In [16]:
SQSAll = SQSAll.reset_index()

In [17]:
SQSAll = SQSAll.fillna(0)

In [18]:
SWCAll[SWCAll['numClicks']==0]['class'].value_counts()

Series([], Name: class, dtype: int64)

In [19]:
SQSAll['class'] = SQSAll['index'].apply(lambda x: 0 if x < 1203 else 1)

In [20]:
SQSAll = SQSAll.drop(columns = ['index', 'query'])

In [21]:
SQSAll

,queryDistance,timeQueries,repeatQueries,repeatClicks,clickDistance,meanClickPosition,numClicks,numClicksPerQuery,numQueries,timeClicks,...,net,org,edu,gov,http,AND,OR,quotes,inter,class
0,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,0,0
1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,0,0
2,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,0,0
3,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,0,0
4,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,1,1
1501,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,1,1
1502,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,1,0,1
1503,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,0,0,0,0,0,0,0,0,1,1


In [22]:
pickle.dump(SWCAll, open( "DataSets/SWCFeatures/SWCFeat.p", "wb" ) )
pickle.dump(SQSAll, open( "DataSets/SQSFeatures/SQSFeat.p", "wb" ) )

In [23]:
SWCAll[SWCAll['numQueries']==0]

,class,cc,cd,dt,ex,fw,in,jj,jjr,jjs,...,numClicksPerQuery,numQueries,timeClicks,uniqueQueries,allSameClicks,uniqueClicks,allSameQueries,queryDistance,timeQueries,repeatQueries
sID,,,,,,,,,,,,,,,,,,,,,
